# Distributed Data Classification with NeMo Curator's `PromptTaskComplexityClassifier`

This notebook demonstrates the use of NeMo Curator's `PromptTaskComplexityClassifier`. The [prompt task and complexity classifier](https://huggingface.co/nvidia/prompt-task-and-complexity-classifier) a multi-headed model which classifies English text prompts across task types and complexity dimensions. It helps with data annotation, which is useful in data blending for foundation model training. Please refer to the NemoCurator Prompt Task and Complexity Classifier Hugging Face page for more information about the prompt task and complexity classifier, including its output labels, here: https://huggingface.co/nvidia/prompt-task-and-complexity-classifier.

Before running this notebook, please see this [Installation Guide](https://docs.nvidia.com/nemo/curator/latest/admin/installation.html#admin-installation) page for instructions on how to install NeMo Curator. Be sure to use an installation method which includes GPU dependencies.

In [1]:
# Silence Ray logs
import os

os.environ["LOGURU_LEVEL"] = "ERROR"

The following imports are required for this tutorial:

In [ ]:
import pandas as pd

from nemo_curator.core.client import RayClient
from nemo_curator.pipeline import Pipeline
from nemo_curator.stages.text.classifiers import PromptTaskComplexityClassifier
from nemo_curator.stages.text.io.reader.jsonl import JsonlReader
from nemo_curator.stages.text.io.writer.jsonl import JsonlWriter

To run a pipeline in NeMo Curator, we must start a Ray cluster. This can be done manually (see the [Ray documentation](https://docs.ray.io/en/latest/ray-core/starting-ray.html)) or with Curator's `RayClient`:

In [3]:
try:
    ray_client = RayClient()
    ray_client.start()
except Exception as e:
    msg = f"Error initializing Ray client: {e}"
    raise RuntimeError(msg) from e

# Initialize Read, Classification, and Write Stages

Functions in NeMo Curator are called stages. For this tutorial, we will initialize 3 stages: a JSONL file reader, a prompt task and complexity classification stage, and a JSONL file writer.

For this tutorial, we will create a sample JSONL file to use:

In [4]:
input_file_path = "./input_data_dir"

# Create sample dataset for the tutorial
text = ["Prompt: Write a Python script that uses a for loop."]
df = pd.DataFrame({"text": text})

try:
    os.makedirs(input_file_path, exist_ok=True)
    df.to_json(input_file_path + "/data.jsonl", orient="records", lines=True)
except Exception as e:
    msg = f"Error creating input file: {e}"
    raise RuntimeError(msg) from e

We can define the reader stage with:

In [5]:
# Read existing directory of JSONL files
read_stage = JsonlReader(input_file_path, files_per_partition=1)

The classifier stage is broken down under the hood into a tokenizer stage and a model inference stage. Tokenization is run on the CPU while model inference is run on the GPU.

In [6]:
# Initialize the prompt task and complexity classifier
classifier_stage = PromptTaskComplexityClassifier()

Finally, we can define a stage for writing the results:

In [7]:
# Write results to a directory
output_file_path = "./prompt_task_complexity_classifier_results"
write_stage = JsonlWriter(output_file_path)

# Initialize Pipeline

In NeMo Curator, we use pipelines to run distributed data workflows using Ray. Pipelines take care of resource allocation and autoscaling to achieve enhanced performance and minimize GPU idleness.

For the distributed data classifiers, we are able to achieve speedups by ensuring that model inference is run in parallel across all available GPUs, while other stages such as I/O, tokenization, and filtering are run across all available CPUs. This is possible because Curator pipelines are composable, which allows each stage in a pipeline to run independently and with its own specified hardware resources.

In [8]:
classifier_pipeline = Pipeline(name="classifier_pipeline", description="Run a classifier pipeline")

# Add stages to the pipeline
classifier_pipeline.add_stage(read_stage)
classifier_pipeline.add_stage(classifier_stage)
classifier_pipeline.add_stage(write_stage)

Pipeline(name='classifier_pipeline', stages=[jsonl_reader(JsonlReader), prompt_task_and_complexity_classifier_classifier(PromptTaskComplexityClassifier), jsonl_writer(JsonlWriter)])

Composability is also what allows a classifier to sit between pre-processing and post-processing stages. Typical text pre-processing add-ons include text normalization (lowercasing, URL/email removal, Unicode cleanup) and language identification and filtering (to keep only target languages). A full pipeline may look something like:

```python
pipeline = Pipeline(name="full_pipeline")
pipeline.add_stage(read_stage)                # reader (JSONL/S3/etc.)
pipeline.add_stage(lang_id_stage)             # optional: language filter
pipeline.add_stage(classifier_stage)          # classifier
pipeline.add_stage(write_stage)               # writer (JSONL/Parquet)
```

# Run the  Classifier

Let's run the full classifier pipeline:

In [ ]:
# Run the pipeline
result = classifier_pipeline.run()

Since the pipeline ran to completion and the result was written to a JSONL file, we can shut down the Ray cluster with:

In [ ]:
try:
    ray_client.stop()
except Exception as e:  # noqa: BLE001
    print(f"Error stopping Ray client: {e}")

# Inspect the Output

The write stage returns a list of written files. We can read the output file as a Pandas DataFrame for inspection.

In [11]:
result_file = result[0].data[0]

result_df = pd.read_json(result_file, lines=True)
result_df.head()

,text,prompt_complexity_score,task_type_1,task_type_2,task_type_prob,creativity_scope,reasoning,contextual_knowledge,number_of_few_shots,domain_knowledge,no_label_reason,constraint_ct
0,Prompt: Write a Python script that uses a for ...,0.27831,Code Generation,Text Generation,0.767,0.0827,0.0632,0.0558,0,0.9802,0,0.5583


We can see that the predictions were generated as expected.